In [2]:
import sknw
import networkx as nx
import pandas as pd
import numpy as np
import os
import sys
import cv2
import skimage.io
import random
import matplotlib.pyplot as plt
import skimage.morphology
from skimage.morphology import skeletonize, remove_small_objects, remove_small_holes, medial_axis
from keras.metrics import MeanIoU

## Mean IoU of post-processed predicted masks
Here we do all of the post-processing steps as described in the image2graph notebook. After that we evaluate the mean IoU.

In [13]:
pred_dir = '/path/to/predictions'
ious = []
m = MeanIoU(num_classes=2)
for i in range(len(os.listdir(pred_dir))):
    pred_path = os.path.join(pred_dir, '{}_pred.png'.format(str(i)))
    gt_path = os.path.join(pred_dir, '{}_ground-truth.png'.format(str(i)))

    pred = cv2.imread(pred_path,1)
    gt = cv2.imread(gt_path,1)
    if np.mean(gt[:,:,-1])==0:
        continue

    gt = cv2.resize(gt, (512,512), interpolation=cv2.INTER_AREA)
    kernel_close = 12
    kernel_open = 3
    kernel_blur = 3
    kernel_close = np.ones((kernel_close,kernel_close), np.uint8)
    kernel_open = np.ones((kernel_open, kernel_open), np.uint8)
    glob_thresh=80
    blur = cv2.medianBlur(pred, kernel_blur)
    glob_thresh_arr = cv2.threshold(blur, glob_thresh, 255, cv2.THRESH_BINARY)[1]
    glob_thresh_arr_smooth = cv2.medianBlur(glob_thresh_arr, kernel_blur)
    mask_thresh = glob_thresh_arr_smooth
    closing = cv2.morphologyEx(mask_thresh, cv2.MORPH_CLOSE, kernel_close)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel_open)
    y_true = gt[:,:,-1]/255
    y_pred = opening[:,:,-1]/255
    m.reset_states()
    m.update_state(y_true,y_pred)

arr = np.array(ious)
print('mean IOU of post-processed masks')
print(np.mean(arr))

mean IOU of post-processed masks
0.40224794380645024


## OTBS metric
Below we evaluate how well we can find optimal paths in our predicted graphs. See short description of this metric in the readme. We also calculate the mean road length in the ground truth graphs and compare it with the mean road length in the predicted graphs. 

In [ ]:
def create_graph(mask_dir, grt=False):
    pred = cv2.imread(mask_dir,1)
    if grt:
        pred = cv2.resize(pred, (512,512), interpolation=cv2.INTER_AREA)
    kernel_close = 12
    kernel_open = 3
    kernel_blur = 3
    kernel_close = np.ones((kernel_close,kernel_close), np.uint8)
    kernel_open = np.ones((kernel_open, kernel_open), np.uint8)
    glob_thresh=80

    blur = cv2.medianBlur(pred, kernel_blur)
    glob_thresh_arr = cv2.threshold(blur, glob_thresh, 255, cv2.THRESH_BINARY)[1]
    glob_thresh_arr_smooth = cv2.medianBlur(glob_thresh_arr, kernel_blur)
    mask_thresh = glob_thresh_arr_smooth
    closing = cv2.morphologyEx(mask_thresh, cv2.MORPH_CLOSE, kernel_close)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel_open)
    medial_int = medial_axis(opening[:,:,-1]).astype(np.uint8)

    graph = sknw.build_sknw(medial_int)
    for component in list(nx.connected_components(graph)):
        if len(component)<3:
            for node in component:
                graph.remove_node(node)
    nn = list(graph)
    vert = len(graph)
    road = graph.size(weight='weight')
    try:
        length, path = nx.single_source_dijkstra(graph, nn[0], nn[-1], weight='weight')
        return length, vert, road
    except:
        return None, vert, road


In [ ]:
pred_dir = '/path/to/predictions'
proposed = []
truths = []
pverts = []
proads = []
tverts = []
troads = []
for i in range(len(os.listdir(pred_dir))):
    pred_path = os.path.join(pred_dir, '{}_pred.png'.format(str(i)))
    gt_path = os.path.join(pred_dir, '{}_ground-truth.png'.format(str(i)))
    prop, pvert, proad = create_graph(pred_path)
    truth, tvert, troad  = create_graph(gt_path, grt=True)
    if truth:
        if prop:
            proposed.append(prop)
            truths.append(truth)
        else:
            proposed.append(2)
            truths.append(1)
    pverts.append(pvert)
    proads.append(proad)
    tverts.append(tvert)
    troads.append(troad)

arr1 = np.array(proposed)
arr2 = np.array(truths)
otbs = np.mean(1-(np.abs((arr1-arr2))/arr2))
arr3 = np.array(pverts)
arr4 = np.array(proads)
arr5 = np.array(tverts)
arr6 = np.array(troads)
print('OTBS-score:')
print(np.mean(otbs))
print('Mean ground-truth road length (sum of edges)')
print(np.mean(arr6))
print('Mean predicted road length (sum of edges)')
print(np.mean(arr4))